In [1]:
# Import library
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
import re
from sklearn.preprocessing import OneHotEncoder
from nltk.util import ngrams
from sklearn.pipeline import Pipeline

import pandas as pd
import simplejson as json
from pprint import pprint
from tqdm import tqdm

c:\Users\bobmst\.conda\envs\ds310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\bobmst\.conda\envs\ds310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\bobmst\.conda\envs\ds310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
df_track_info_complete = pd.read_pickle('../data/complete_feature.pkl')
df_track_info_complete

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
uri,,,,,,,,,,,,,,,,,,,,,
spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,0.000000,0.912210,0.813,0.363636,0.842733,0.0,0.125649,0.031225,0.006977,0.0471,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
spotify:track:6I9VzXrHxO9rA9A5euc8Ak,0.000006,0.781029,0.838,0.454545,0.893573,0.0,0.118380,0.025000,0.025025,0.2420,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
spotify:track:0WqIKmW4BTrj3eJFmnCKMv,0.000012,0.670030,0.758,0.181818,0.851050,0.0,0.218069,0.002390,0.000000,0.0598,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
spotify:track:1AWQoqb9bSvzTjaLralEkT,0.000018,0.900101,0.714,0.363636,0.859462,0.0,0.146417,0.201807,0.000234,0.0521,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
spotify:track:1lzr43nnXAijIGYnCT8M8H,0.000024,0.860747,0.606,0.000000,0.882707,1.0,0.074039,0.056325,0.000000,0.3130,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
spotify:track:4cKPzfOUIZWXhu3GDnESjL,0.999976,0.556004,0.822,1.000000,0.844119,0.0,0.063759,0.001546,0.000000,0.2150,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
spotify:track:0VICzaRdctD8XGfDstBjQY,0.999982,0.484359,0.761,0.545455,0.882197,1.0,0.038110,0.120482,0.000000,0.2960,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
spotify:track:0B4YuevrDNhh71edZrFiOs,0.999988,0.405651,0.767,1.000000,0.848899,0.0,0.177570,0.000078,0.000003,0.3930,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [3]:
files = ['mpd.slice.0-999.json'
,'mpd.slice.1000-1999.json'
,'mpd.slice.2000-2999.json'
,'mpd.slice.3000-3999.json'
,'mpd.slice.4000-4999.json'
,'mpd.slice.5000-5999.json'
,'mpd.slice.6000-6999.json'
,'mpd.slice.7000-7999.json'
,'mpd.slice.8000-8999.json'
,'mpd.slice.9000-9999.json']

In [4]:
def get_sigle_playlist_file(filename):
    with open(filename,'r') as f:
        data = json.load(f)
    return pd.DataFrame(data['playlists'])

In [6]:
get_sigle_playlist_file("../data/"+f_name)

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description
0,Throwbacks,false,0,1493424000,52,47,1,"[{'pos': 0, 'artist_name': 'Missy Elliott', 't...",6,11532414,37,NaN
1,Awesome Playlist,false,1,1506556800,39,23,1,"[{'pos': 0, 'artist_name': 'Survivor', 'track_...",5,11656470,21,NaN
2,korean,false,2,1505692800,64,51,1,"[{'pos': 0, 'artist_name': 'Hoody', 'track_uri...",18,14039958,31,NaN
3,mat,false,3,1501027200,126,107,1,"[{'pos': 0, 'artist_name': 'Camille Saint-Saën...",4,28926058,86,NaN
4,90s,false,4,1401667200,17,16,2,"[{'pos': 0, 'artist_name': 'The Smashing Pumpk...",7,4335282,16,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
995,old,false,995,1507852800,41,40,1,"[{'pos': 0, 'artist_name': 'Katrina', 'track_u...",8,9917901,36,NaN
996,Daze,false,996,1479254400,17,17,1,"[{'pos': 0, 'artist_name': 'PARTYNEXTDOOR', 't...",13,3699248,15,NaN
997,rap,false,997,1410307200,119,98,1,"[{'pos': 0, 'artist_name': 'LoveRance', 'track...",63,27538723,82,NaN
998,Country,false,998,1507939200,108,75,1,"[{'pos': 0, 'artist_name': 'Hunter Hayes', 'tr...",37,24950143,40,NaN


In [9]:
ls_df_playlists = []
for f_name in tqdm(files):
    ls_df_playlists.append(get_sigle_playlist_file("../data/"+f_name))
df_playlists = pd.concat(ls_df_playlists, ignore_index=True)
df_playlists

100%|██████████| 10/10 [00:01<00:00,  5.02it/s]


,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description
0,Throwbacks,false,0,1493424000,52,47,1,"[{'pos': 0, 'artist_name': 'Missy Elliott', 't...",6,11532414,37,NaN
1,Awesome Playlist,false,1,1506556800,39,23,1,"[{'pos': 0, 'artist_name': 'Survivor', 'track_...",5,11656470,21,NaN
2,korean,false,2,1505692800,64,51,1,"[{'pos': 0, 'artist_name': 'Hoody', 'track_uri...",18,14039958,31,NaN
3,mat,false,3,1501027200,126,107,1,"[{'pos': 0, 'artist_name': 'Camille Saint-Saën...",4,28926058,86,NaN
4,90s,false,4,1401667200,17,16,2,"[{'pos': 0, 'artist_name': 'The Smashing Pumpk...",7,4335282,16,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,rap,false,9995,1491782400,34,26,1,"[{'pos': 0, 'artist_name': 'Lecrae', 'track_ur...",16,8530582,15,NaN
9996,Blues,false,9996,1482364800,57,48,1,"[{'pos': 0, 'artist_name': 'Robert Johnson', '...",15,13010049,30,NaN
9997,game songs,false,9997,1508371200,27,24,4,"[{'pos': 0, 'artist_name': 'NateWantsToBattle'...",23,5104068,7,NaN
9998,country,false,9998,1466208000,12,12,1,"[{'pos': 0, 'artist_name': 'Little Big Town', ...",3,2459585,12,NaN


In [21]:
df_playlists.iloc[0,:]['tracks']

[{'pos': 0,
  'artist_name': 'Missy Elliott',
  'track_uri': 'spotify:track:0UaMYEvWZi0ZqiDOoHU3YI',
  'artist_uri': 'spotify:artist:2wIVse2owClT7go1WT98tk',
  'track_name': 'Lose Control (feat. Ciara & Fat Man Scoop)',
  'album_uri': 'spotify:album:6vV5UrXcfyQD1wu4Qo2I9K',
  'duration_ms': 226863,
  'album_name': 'The Cookbook'},
 {'pos': 1,
  'artist_name': 'Britney Spears',
  'track_uri': 'spotify:track:6I9VzXrHxO9rA9A5euc8Ak',
  'artist_uri': 'spotify:artist:26dSoYclwsYLMAKD3tpOr4',
  'track_name': 'Toxic',
  'album_uri': 'spotify:album:0z7pVBGOD7HCIB7S8eLkLI',
  'duration_ms': 198800,
  'album_name': 'In The Zone'},
 {'pos': 2,
  'artist_name': 'Beyoncé',
  'track_uri': 'spotify:track:0WqIKmW4BTrj3eJFmnCKMv',
  'artist_uri': 'spotify:artist:6vWDO969PvNqNYHIOW5v0m',
  'track_name': 'Crazy In Love',
  'album_uri': 'spotify:album:25hVFAxTlDvXbx2X2QkUkE',
  'duration_ms': 235933,
  'album_name': 'Dangerously In Love (Alben für die Ewigkeit)'},
 {'pos': 3,
  'artist_name': 'Justin Timb

In [22]:
# df_playlists['tracks']

def get_track_id_in_playlist(playlist):
    return [x['track_uri'] for x in playlist['tracks']]

In [35]:
df_playlists_songs = df_playlists.apply(get_track_id_in_playlist,axis=1).to_frame(name='tracks')
df_playlists_songs.set_index(df_playlists['pid'])
df_playlists_songs

,tracks
0,"[spotify:track:0UaMYEvWZi0ZqiDOoHU3YI, spotify..."
1,"[spotify:track:2HHtWyy5CgaQbC7XSoOb0e, spotify..."
2,"[spotify:track:74tqql9zP6JjF5hjkHHUXp, spotify..."
3,"[spotify:track:4WJ7UMD4i6DOPzyXU5pZSz, spotify..."
4,"[spotify:track:4iCGSi1RonREsPtfEKYj5b, spotify..."
...,...
9995,"[spotify:track:0hAUGZg91bjp2isiMCr1of, spotify..."
9996,"[spotify:track:7zSKmt2QD3yf4Rkb2MSuAf, spotify..."
9997,"[spotify:track:46ZY3cPpxVCMShhdkOD8gE, spotify..."
9998,"[spotify:track:5pD1qwHvnKFcQ2qjwsoKYF, spotify..."


In [10]:
# ls_playlists = []
# for playlist in data['playlists']:
#     ls_playlists.append({
#         'pid': playlist['pid'],
#         'tracks': [x['track_uri'] for x in playlist['tracks']]
#     })
# df_playlists_songs = pd.DataFrame(ls_playlists)
# df_playlists_songs

NameError: name 'data' is not defined

## summarize the feature of all music within a playlist

In [36]:
def get_features_summary(x):
    temp = df_track_info_complete.loc[x['tracks'],:].mean(axis=0)
    # temp['id'] = x['pid']
    return temp

In [38]:
df_playlists_scores = df_playlists_songs.apply(get_features,axis=1)
df_playlists_scores

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.000148,0.670108,0.781077,0.458042,0.878004,0.692308,0.107682,0.084010,0.000675,0.187087,...,0.000000,0.096154,0.750000,0.153846,0.153846,0.846154,0.000000,0.019231,0.826923,0.153846
1,0.000412,0.496854,0.695923,0.405594,0.826754,0.538462,0.094507,0.162879,0.223932,0.179344,...,0.000000,0.076923,0.717949,0.205128,0.128205,0.871795,0.000000,0.051282,0.846154,0.102564
2,0.000714,0.677157,0.692953,0.454545,0.878253,0.515625,0.100130,0.270181,0.000638,0.168894,...,0.015625,0.046875,0.859375,0.093750,0.187500,0.812500,0.000000,0.078125,0.750000,0.171875
3,0.001273,0.519100,0.620902,0.463925,0.802684,0.714286,0.069574,0.274613,0.203359,0.188278,...,0.007937,0.126984,0.714286,0.158730,0.166667,0.833333,0.000000,0.103175,0.761905,0.134921
4,0.001693,0.581469,0.650418,0.304813,0.834297,0.823529,0.042801,0.177901,0.081841,0.166524,...,0.000000,0.058824,0.823529,0.117647,0.117647,0.882353,0.000000,0.000000,0.823529,0.176471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.224871,0.657951,0.764588,0.403743,0.872175,0.676471,0.261862,0.167604,0.004311,0.212762,...,0.000000,0.088235,0.852941,0.058824,0.147059,0.852941,0.000000,0.000000,0.882353,0.117647
9996,0.771463,0.578770,0.496304,0.499203,0.791370,0.631579,0.071261,0.616040,0.072902,0.215796,...,0.000000,0.105263,0.631579,0.263158,0.315789,0.684211,0.000000,0.035088,0.578947,0.385965
9997,0.705518,0.518182,0.858556,0.491582,0.873669,0.222222,0.096081,0.090564,0.000058,0.210326,...,0.000000,0.037037,0.962963,0.000000,0.148148,0.851852,0.000000,0.111111,0.814815,0.074074
9998,0.027274,0.565843,0.733833,0.651515,0.862283,1.000000,0.038837,0.178882,0.000002,0.233925,...,0.000000,0.083333,0.833333,0.083333,0.166667,0.833333,0.000000,0.083333,0.833333,0.083333


## Calculate Cos Similarity

### Single Song

suppose the user had listend one song and we need to give him a recommendation of a playlist

In [39]:
one_song = df_track_info_complete.iloc[0,:]
one_song

0     0.000000
1     0.912210
2     0.813000
3     0.363636
4     0.842733
5     0.000000
6     0.125649
7     0.031225
8     0.006977
9     0.047100
10    0.810000
11    0.532108
12    0.042414
13    0.000000
14    1.000000
15    0.000000
16    0.000000
17    1.000000
18    0.000000
19    0.000000
20    1.000000
21    0.000000
22    0.000000
23    1.000000
24    0.000000
Name: spotify:track:0UaMYEvWZi0ZqiDOoHU3YI, dtype: float64

In [42]:
cosine_similarity(df_playlists_scores,one_song.values.reshape(1,-1))

array([[0.93667293],
       [0.93118074],
       [0.95199154],
       ...,
       [0.93553906],
       [0.89281844],
       [0.91524884]])

In [50]:
def recommend_playlist_by_feature(df_playlists_scores,features,df_playlists,n=10):
    df_recommendation = df_playlists.copy()
    df_recommendation['cos_sim'] = cosine_similarity(df_playlists_scores,features.values.reshape(1,-1))
    return df_recommendation.sort_values('cos_sim',ascending=False).head(n)

In [51]:
recommend_playlist_by_feature(df_playlists_scores,one_song,df_playlists)

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description,cos_sim
425,Phone,false,425,1507593600,10,10,1,"[{'pos': 0, 'artist_name': 'Mickey Singh', 'tr...",3,2339579,7,NaN,0.988541
4815,J Balvin,false,4815,1500076800,17,9,1,"[{'pos': 0, 'artist_name': 'J Balvin', 'track_...",6,3539594,5,NaN,0.985485
313,Spanish,false,313,1501804800,6,4,1,"[{'pos': 0, 'artist_name': 'Alvaro Soler', 'tr...",4,1212478,4,NaN,0.984462
6488,favorite,false,6488,1508457600,17,17,1,"[{'pos': 0, 'artist_name': 'Khalid', 'track_ur...",7,3514090,17,NaN,0.984095
1050,Bhangra,false,1050,1460764800,84,72,4,"[{'pos': 0, 'artist_name': 'Bally Sagoo', 'tra...",23,21808612,62,NaN,0.981201
6370,💙💙,false,6370,1505952000,36,32,1,"[{'pos': 0, 'artist_name': 'Shakira', 'track_u...",8,7856445,23,NaN,0.980650
6382,Mam,false,6382,1506643200,18,17,2,"[{'pos': 0, 'artist_name': 'J Balvin', 'track_...",6,3843518,12,NaN,0.979721
3734,Spanish music,false,3734,1508457600,14,13,1,"[{'pos': 0, 'artist_name': 'Maluma', 'track_ur...",5,3138298,11,NaN,0.978807
1142,Latin,false,1142,1389744000,76,57,1,"[{'pos': 0, 'artist_name': 'Fruko Y Sus Tesos'...",23,17121689,32,NaN,0.977956
5707,Oldies,false,5707,1508457600,7,7,1,"[{'pos': 0, 'artist_name': 'Mary J. Blige', 't...",4,1665678,7,NaN,0.977732


### multiple songs

In [52]:
mimic_play_list = df_track_info_complete.iloc[[10,30,55],:]

In [55]:
recommend_playlist_by_feature(df_playlists_scores,mimic_play_list.mean(axis=0),df_playlists)

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description,cos_sim
746,EBM,false,746,1507766400,21,15,1,"[{'pos': 0, 'artist_name': 'Nitzer Ebb', 'trac...",12,5825144,12,NaN,0.982877
6750,Sleep,false,6750,1455062400,33,32,1,"[{'pos': 0, 'artist_name': 'The Rolling Stones...",4,8080274,26,NaN,0.981952
4980,workout,false,4980,1442534400,7,7,1,"[{'pos': 0, 'artist_name': 'Avenged Sevenfold'...",3,1897303,6,NaN,0.977384
3913,summer,false,3913,1495152000,12,8,1,"[{'pos': 0, 'artist_name': 'Ed Sheeran', 'trac...",3,2698524,5,NaN,0.977222
718,Swing,false,718,1494460800,24,11,1,"[{'pos': 0, 'artist_name': 'Alice Francis', 't...",3,5718853,6,NaN,0.975678
1948,Rock,false,1948,1446681600,18,9,1,"[{'pos': 0, 'artist_name': 'Crown The Empire',...",6,4278631,6,NaN,0.974361
1837,Chill,false,1837,1504828800,10,9,3,"[{'pos': 0, 'artist_name': 'Panic! At The Disc...",7,2257804,9,NaN,0.974066
3338,MY mixtape,false,3338,1470009600,32,22,1,"[{'pos': 0, 'artist_name': 'Goblins from Mars'...",4,6631283,13,NaN,0.973890
1290,The good stuff,false,1290,1485561600,8,7,1,"[{'pos': 0, 'artist_name': 'deadmau5', 'track_...",2,2205557,5,NaN,0.973751
2283,amor,false,2283,1446940800,19,16,1,"[{'pos': 0, 'artist_name': 'Washed Out', 'trac...",3,3854221,14,NaN,0.972923
